In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import datetime

# <font color=red>防止过拟合</font>
* 增加数据
* 整合化
* Dropout方法

In [ ]:
# 载入数据
mnist = input_data.read_data_sets('mnist_data',one_hot=True)

# 每个批次的大小
batch_size = 50
# 计算一共有多少个批次,通过整除算出
n_batch = mnist.train.num_examples // batch_size
# dropout比例
keep_prob_value = 0.7
# 定义占位符
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

# 创建一个简单的神经网络，784 * 128 *28 * 10

#定义第1个隐藏层
w1 = tf.Variable( tf.truncated_normal( [784,2000],stddev=0.1 ) )
b1 = tf.Variable(tf.zeros([1,2000]) + 0.1)
net1 = tf.matmul(x,w1) + b1
L1 = tf.nn.tanh(net1)
L1_dropout = tf.nn.dropout(L1,keep_prob)

#定义第2个隐藏层
w2 = tf.Variable( tf.truncated_normal([2000,1800],stddev=0.1) )
b2 = tf.Variable(tf.zeros([1,1800]))
net2 = tf.matmul(L1_dropout,w2) + b2
L2 = tf.nn.tanh(net2)
L2_dropout = tf.nn.dropout(L2,keep_prob)

#定义第3个隐藏层
w3 = tf.Variable(tf.truncated_normal([1800,2000],stddev=0.1))
b3 = tf.Variable(tf.zeros([1,2000]) + 0.1)
net3 = tf.matmul(L2_dropout,w3) + b3
L3 = tf.nn.tanh(net3)
L3_dropout = tf.nn.dropout(L3,keep_prob)

# 定义一个神经网络的输出层
w4 = tf.Variable(tf.random_normal([2000,10]))
b4 = tf.Variable(tf.random_normal([1,10]))
net4 = tf.matmul(L3_dropout,w4) + b4
prediction = tf.nn.softmax(net4)


# 使用交叉熵函数为损失函数，交叉熵的导数与激活函数无关，与误差相关，利于梯度下降
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
# 梯度下降优化器,步长为0.1
# optimizer = tf.train.GradientDescentOptimizer(0.02)
# optimizer = tf.train.
optimizer = tf.train.MomentumOptimizer(learning_rate=0.02,momentum=0.7)
# 最小化目标函数
with tf.device('/gpu:0'):
    objectFun = optimizer.minimize(loss)

#比较预测值与真实值，并将结果保持在一个布尔列表中
#tf.argmax 返回的是vector中的最大值的索引号
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))

# 求准确率,tf.cast数据类型转换
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

begintime = datetime.datetime.now()

config = tf.ConfigProto()  
config.gpu_options.per_process_gpu_memory_fraction = 0.5  # 程序最多只能占用指定gpu50%的显存  
config.gpu_options.allow_growth = True      #程序按需申请内存  
sess = tf.Session(config = config) 
# 迭代
with tf.Session(config=config ) as sess:
    
    # 初始化变量
    sess.run(tf.global_variables_initializer())
    # 进行迭代，训练神经网络
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(objectFun,feed_dict={x:batch_xs,y:batch_ys,keep_prob:keep_prob_value})
        
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:keep_prob_value})
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:keep_prob_value})
        print('迭代',str(epoch),', Testing Accuacy',test_acc,'Training Accuacy',train_acc)
endtime = datetime.datetime.now()
print('耗时：',(endtime - begintime).seconds,'秒')